In [38]:
import itertools
import pandas as pd
import numpy as np
import re
import os
from tqdm import tqdm
import csv
import pandas as pd
# Drawing the embeddings
import matplotlib.pyplot as plt
from keras.preprocessing.text import Tokenizer

# Deep learning: 
from keras.models import Input, Model
from keras.layers import Dense

from scipy import sparse

import re
# import gensim.models.keyedvectors as word2vec
from gensim.test.utils import datapath
from gensim.models import KeyedVectors

import pprint

In [2]:
def create_new_lines(dataset: list, item: str):
    item = item.replace(",", "")
    item = re.sub(r"e.g.", "", item)
    item = re.sub(r"i.e.", "", item)
    item = re.sub(r"et al.", "", item)
    item = re.sub(r"i.e.", "", item)
    item = re.sub(r"i.e.,", "", item)
    item = re.sub(r"vs.,", "", item)
    item = re.sub(r"vs.", "", item)
    item = re.sub(r"i. e.", "", item)
    lines = item.split(". ")
    dataset = [dataset.append(line) or line for line in lines]

In [3]:
def text_preprocessing(
    text:list,
    punctuations = r'''!()-[]{};:'"\,<>./?@#$%^&*_“~+''',
    stop_words=['and', 'a', 'is', 'the', 'in', 'be', 'will']
    )->list:
    """
    A method to preproces text
    """
    for x in text.lower(): 
        if x in punctuations: 
            text = text.replace(x, "")

    # Removing words that have numbers in them
    text = re.sub(r'\w*\d\w*', '', text)

    # Removing digits
    text = re.sub(r'[0-9]+', '', text)

    # Cleaning the whitespaces
    text = re.sub(r'\s+', ' ', text).strip()

    # Setting every word to lower
    text = text.lower()

    # Converting all our text to a list 
    text = text.split(' ')

    # Droping empty strings
    text = [x for x in text if x!='']

    # Droping stop words
    text = [x for x in text if x not in stop_words]

    return text

In [4]:
def preprocess(dataset: list)->list:
    processed = []
    for line in dataset:
        text = text_preprocessing(line)
        processed.append(text)
    return processed    

In [5]:
def createContextDict(line: list, all_words: list):
    for i, word in enumerate(line):
        all_words.append(word)
        for w in range(window):
            # Getting the context that is ahead by *window* words
            if i + 1 + w < len(line): 
                word_lists.append([word] + [line[(i + 1 + w)]])
            # Getting the context that is behind by *window* words    
            if i - w - 1 >= 0:
                word_lists.append([word] + [line[(i - w - 1)]])
        

In [6]:
def create_unique_word_dict(text:list) -> dict:
    """
    A method that creates a dictionary where the keys are unique words
    and key values are indices
    """
    # Getting all the unique words from our text and sorting them alphabetically
    words = list(set(text))
    words.sort()

    # Creating the dictionary for the unique words
    unique_word_dict = {}
    for i, word in enumerate(words):
        unique_word_dict.update({
            word: i
        })

    return unique_word_dict  

In [7]:
dataset = []
word_lists = []
all_processed_text = []

word_lists = []
# Defining the window for context
window = 4
all_words = []

file1 = open('data/arxiv_3.txt', 'r') 
dataset = file1.readlines()

# count = 0
# for line in fileContent: 
#     count += 1
#     create_new_lines(dataset, line)
    
processed_dataset = preprocess(dataset)

for line in processed_dataset:
    createContextDict(line, all_words)
    

In [8]:
print(processed_dataset)

[['xlnet', 'generalized', 'autoregressive', 'pretraining', 'language', 'understanding'], ['modeling', 'bidirectional', 'contexts', 'denoising', 'autoencoding', 'pretraining', 'bert', 'autoregressive', 'language', 'modeling'], ['masks', 'bert', 'pretrainfinetune', 'discrepancy'], ['xlnet', 'generalized', 'autoregressive', 'pretraining', 'learning', 'bidirectional', 'contexts', 'bert', 'autoregressive', 'formulation'], ['xlnet', 'ransformerxl', 'autoregressive', 'model', 'into', 'pretraining'], ['xlnet', 'outperforms', 'bert', 'question', 'answering', 'natural', 'language', 'inference', 'sentiment', 'analysis', 'document', 'ranking'], ['bert', 'transformer', 'stack', 'of', 'lstms'], ['roberta', 'robustly', 'optimized', 'bert', 'bert'], ['bert', 'undertrained', 'stateoftheart', 'results', 'glue', 'race', 'squad', 'bert', 'undertrained', 'improved', 'training', 'bert', 'models', 'roberta', 'postbert'], ['ordered', 'neurons', 'recurrent', 'neural', 'networks', 'rnn'], ['natural', 'language'

In [11]:
unique_word_dict = create_unique_word_dict(all_words)

In [9]:
print(len(all_words))

1970


In [12]:
# Defining the number of features (unique words)
n_words = len(unique_word_dict)

# Getting all the unique words 
words = list(unique_word_dict.keys())

# Creating the X and Y matrices using one hot encoding
X = []
Y = []

In [13]:
for i, word_list in tqdm(enumerate(word_lists)):
    # Getting the indices
    main_word_index = unique_word_dict.get(word_list[0])
    context_word_index = unique_word_dict.get(word_list[1])

    # Creating the placeholders   
    X_row = np.zeros(n_words)
    Y_row = np.zeros(n_words)

    # One hot encoding the main word
    X_row[main_word_index] = 1

    # One hot encoding the Y matrix words 
    Y_row[context_word_index] = 1

    # Appending to the main matrices
    X.append(X_row)
    Y.append(Y_row)

11550it [00:00, 94657.35it/s]


In [14]:
# Converting the matrices into a sparse format because the vast majority of the data are 0s
X = sparse.csr_matrix(X)
Y = sparse.csr_matrix(Y)

# Defining the size of the embedding
embed_size = 5

In [15]:
# Defining the neural network
inp = Input(shape=(X.shape[1],))
x = Dense(units=embed_size, activation='linear')(inp)
x = Dense(units=Y.shape[1], activation='softmax')(x)
model = Model(inputs=inp, outputs=x)
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam')

# Optimizing the network weights
model.fit(
    x=X, 
    y=Y, 
    batch_size=256,
    epochs=1000
    )


Epoch 1/1000
11550/11550 [==============================] - 0s 18us/step - loss: 6.6374
Epoch 2/1000
11550/11550 [==============================] - 0s 11us/step - loss: 6.6039
Epoch 3/1000
11550/11550 [==============================] - 0s 11us/step - loss: 6.5435
Epoch 4/1000
11550/11550 [==============================] - 0s 11us/step - loss: 6.4487
Epoch 5/1000
11550/11550 [==============================] - 0s 10us/step - loss: 6.3318
Epoch 6/1000
11550/11550 [==============================] - 0s 11us/step - loss: 6.2137
Epoch 7/1000
11550/11550 [==============================] - 0s 10us/step - loss: 6.1134
Epoch 8/1000
11550/11550 [==============================] - 0s 11us/step - loss: 6.0408
Epoch 9/1000
11550/11550 [==============================] - 0s 10us/step - loss: 5.9978
Epoch 10/1000
11550/11550 [==============================] - 0s 11us/step - loss: 5.9751
Epoch 11/1000
11550/11550 [==============================] - 0s 10us/step - loss: 5.9634
Epoch 12/1000
11550/11550 [==

11550/11550 [==============================] - 0s 11us/step - loss: 4.9226
Epoch 91/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.9100
Epoch 92/1000
11550/11550 [==============================] - 0s 11us/step - loss: 4.8977
Epoch 93/1000
11550/11550 [==============================] - 0s 11us/step - loss: 4.8858
Epoch 94/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.8740
Epoch 95/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.8627
Epoch 96/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.8514
Epoch 97/1000
11550/11550 [==============================] - 0s 11us/step - loss: 4.8403
Epoch 98/1000
11550/11550 [==============================] - 0s 15us/step - loss: 4.8296
Epoch 99/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.8191
Epoch 100/1000
11550/11550 [==============================] - 0s 11us/step - loss: 4.8088
Epoch 101/1000
11550/11550 [======

11550/11550 [==============================] - 0s 14us/step - loss: 4.3755
Epoch 182/1000
11550/11550 [==============================] - 0s 11us/step - loss: 4.3728
Epoch 183/1000
11550/11550 [==============================] - 0s 11us/step - loss: 4.3701
Epoch 184/1000
11550/11550 [==============================] - 0s 11us/step - loss: 4.3673
Epoch 185/1000
11550/11550 [==============================] - 0s 11us/step - loss: 4.3646
Epoch 186/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.3620
Epoch 187/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.3592
Epoch 188/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.3568
Epoch 189/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.3540
Epoch 190/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.3516
Epoch 191/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.3491
Epoch 192/1000
11550/11550 [

11550/11550 [==============================] - 0s 11us/step - loss: 4.2068
Epoch 273/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.2055
Epoch 274/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.2042
Epoch 275/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.2030
Epoch 276/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.2018
Epoch 277/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.2007
Epoch 278/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.1995
Epoch 279/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.1983
Epoch 280/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.1972
Epoch 281/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.1960
Epoch 282/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.1949
Epoch 283/1000
11550/11550 

11550/11550 [==============================] - 0s 10us/step - loss: 4.1207
Epoch 364/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.1197
Epoch 365/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.1190
Epoch 366/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.1184
Epoch 367/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.1176
Epoch 368/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.1169
Epoch 369/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.1162
Epoch 370/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.1153
Epoch 371/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.1147
Epoch 372/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.1141
Epoch 373/1000
11550/11550 [==============================] - 0s 9us/step - loss: 4.1135
Epoch 374/1000
11550/11550 [===

...

11550/11550 [==============================] - 0s 10us/step - loss: 4.0024
Epoch 637/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0021
Epoch 638/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0022
Epoch 639/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0017
Epoch 640/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0015
Epoch 641/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0011
Epoch 642/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0009
Epoch 643/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0005
Epoch 644/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0003
Epoch 645/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0001
Epoch 646/1000
11550/11550 [==============================] - 0s 10us/step - loss: 4.0002
Epoch 647/1000
11550/1155

11550/11550 [==============================] - 0s 10us/step - loss: 3.9820
Epoch 728/1000
11550/11550 [==============================] - 0s 11us/step - loss: 3.9818
Epoch 729/1000
11550/11550 [==============================] - 0s 10us/step - loss: 3.9819
Epoch 730/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9816
Epoch 731/1000
11550/11550 [==============================] - 0s 10us/step - loss: 3.9813
Epoch 732/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9811
Epoch 733/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9809
Epoch 734/1000
11550/11550 [==============================] - 0s 10us/step - loss: 3.9805
Epoch 735/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9804
Epoch 736/1000
11550/11550 [==============================] - 0s 10us/step - loss: 3.9803
Epoch 737/1000
11550/11550 [==============================] - 0s 10us/step - loss: 3.9803
Epoch 738/1000
11550/11550 [=

11550/11550 [==============================] - 0s 9us/step - loss: 3.9660
Epoch 819/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9661
Epoch 820/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9659
Epoch 821/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9657
Epoch 822/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9657
Epoch 823/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9653
Epoch 824/1000
11550/11550 [==============================] - 0s 10us/step - loss: 3.9654
Epoch 825/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9652
Epoch 826/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9650
Epoch 827/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9648
Epoch 828/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9646
Epoch 829/1000
11550/11550 [=======

11550/11550 [==============================] - 0s 9us/step - loss: 3.9533
Epoch 910/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9531
Epoch 911/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9530
Epoch 912/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9530
Epoch 913/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9526
Epoch 914/1000
11550/11550 [==============================] - 0s 10us/step - loss: 3.9526
Epoch 915/1000
11550/11550 [==============================] - 0s 10us/step - loss: 3.9525
Epoch 916/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9523
Epoch 917/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9522
Epoch 918/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9523
Epoch 919/1000
11550/11550 [==============================] - 0s 9us/step - loss: 3.9522
Epoch 920/1000
11550/11550 [======

11550/11550 [==============================] - 0s 10us/step - loss: 3.9425


In [17]:
# Obtaining the weights from the neural network. 
# These are the so called word embeddings

# The input layer 
weights = model.get_weights()[0]

# Creating a dictionary to store the embeddings in. The key is a unique word and 
# the value is the numeric vector
embedding_dict = {}
for word in words: 
    embedding_dict.update({
        word: weights[unique_word_dict.get(word)]
        })

In [18]:
# Ploting the embeddings
plt.figure(figsize=(200, 200))
for word in list(unique_word_dict.keys()):
    coord = embedding_dict.get(word)
    plt.scatter(coord[0], coord[1])
    plt.annotate(word, (coord[0], coord[1]))       

In [21]:
tokenizer = Tokenizer(filters='''!()-[]{};:'"\,<>./?@#$%^&*_“~+''')
tokenizer.fit_on_texts(processed_dataset)
corpus = tokenizer.texts_to_sequences(processed_dataset)

In [23]:
vectors = model.get_weights()[0]

In [24]:
vectors

array([[ 0.77536285, -7.704255  ,  0.6369099 ,  0.13260293,  1.1477503 ],
       [-0.16279025, -3.1027749 , -2.6633997 ,  1.9186282 ,  0.5944619 ],
       [ 1.8247057 ,  0.8843093 , -1.041461  ,  1.1601079 ,  0.5902169 ],
       ...,
       [-3.3290248 , -1.8556453 , -3.000834  ,  1.4016578 , -1.2977673 ],
       [ 5.7440605 ,  2.5621662 , -4.7216845 ,  1.249307  ,  2.2630994 ],
       [ 0.5830086 ,  1.5998446 ,  6.4593544 ,  1.0984    ,  0.58541405]],
      dtype=float32)

In [36]:
vectors = model.get_weights()[0]
with open('vectors_new.txt', 'w') as f:
    f.write(' '.join([str(700), str(200)]))
    f.write('\n')
    for word in words:
        f.write(word)
        f.write(' ')
        f.write(' '.join(map(str, list(weights[unique_word_dict.get(word)]))))
        f.write('\n')

In [39]:
w2v = KeyedVectors.load_word2vec_format('./vectors.txt', binary=False)
pprint.pprint(w2v.most_similar(positive=['bert']))

[('bidirectional', 0.6121548414230347),
 ('inference', 0.5848471522331238),
 ('language', 0.5712134838104248),
 ('natural', 0.568121075630188),
 ('pretrained', 0.5610018968582153),
 ('pretraining', 0.5552810430526733),
 ('processing', 0.5532826781272888),
 ('autoregressive', 0.5525839328765869),
 ('nlp', 0.5367773771286011),
 ('ranking', 0.5365626215934753)]


In [ ]:
print(len(all_words))

In [ ]:
print(len(unique_word_dict))

In [ ]:
print(unique_word_dict)

In [ ]:
df = pd.DataFrame(dataset, columns = ['Text'])
df.to_csv("arxiv.csv")

In [ ]:
ser_lower = df['Text'].str.lower()

In [ ]:
df = pd.DataFrame(ser_lower)

In [ ]:
df.head()